# Extract Frames from Videos


In [ ]:
import cv2
#video_name = "awarpur/4.mp4" # or any other extension like .avi etc
vidcap = cv2.VideoCapture(video_name)
success,image = vidcap.read()
count = 3067
while success:
  cv2.imwrite("frames/vid1/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

# Model inferencing


In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

torch.cuda.set_per_process_memory_fraction(0.1, device=0)
class ObjectDetection:

    def __init__(self, capture_index):
       
        self.capture_index = capture_index
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        
        self.CLASS_NAMES_DICT = self.model.model.names
    
        self.box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)
    

    def load_model(self):
       
        model = YOLO("M:/CV/ACC_POC/weights/yolo8n_100epochs.pt")  # load a pretrained YOLOv8n model
        model.fuse()
    
        return model


    def predict(self, frame):
       
        results = self.model(frame)
        
        return results
    

    def plot_bboxes(self, results, frame):
        
        xyxys = []
        confidences = []
        class_ids = []
        
        # Extract detections for person class
        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            
            if class_id == 0:
                
                xyxys.append(result.boxes.xyxy.cpu().numpy())
                confidences.append(result.boxes.conf.cpu().numpy())
                class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
            
        
        # Setup detections for visualization
        detections = Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )
        
    
        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
        for _, confidence, class_id, tracker_id
        in detections]
        
        # Annotate and display frame
        frame = self.box_annotator.annotate(frame=frame, detections=detections, labels=self.labels)
        
        return frame
    
    
    
    def __call__(self):

        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
      
        while True:
          
            start_time = time()
            
            ret, frame = cap.read()
            assert ret
            
            results = self.predict(frame)
            frame = self.plot_bboxes(results, frame)
            
            end_time = time()
            fps = 1/np.round(end_time - start_time, 2)
             
            cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
            
            cv2.imshow('YOLOv8 Detection', frame)
 
            if cv2.waitKey(5) & 0xFF == 27:
                
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
        
    
#detector = ObjectDetection(capture_index=0)
detector = ObjectDetection(capture_index='M:/CV/ACC_POC/videos/4.mp4')
detector()

#'M:/CV/ACC_POC/videos/4.mp4'

# Detect obj and predict result

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator


class ObjectDetection:

    def __init__(self, capture_index):
       
        self.capture_index = capture_index
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        
        self.CLASS_NAMES_DICT = self.model.model.names
    
        self.box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)
    

    def load_model(self):
       
        model = YOLO("M:/CV/ACC_POC/weights/best.pt")  # load a pretrained YOLOv8n model
        model.fuse()
    
        return model


    def predict(self, frame):
       
        results = self.model(frame)
        
        return results
    

    def plot_bboxes(self, results, frame):
        
        xyxys = []
        confidences = []
        class_ids = []
        
        # Extract detections for person class
        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            
            if class_id == 0:
                
                xyxys.append(result.boxes.xyxy.cpu().numpy())
                confidences.append(result.boxes.conf.cpu().numpy())
                class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
            
        
        # Setup detections for visualization
        detections = Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )
        
    
        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
        for _, confidence, class_id, tracker_id
        in detections]
        
        # Annotate and display frame
        frame = self.box_annotator.annotate(frame=frame, detections=detections, labels=self.labels)
        
        return frame
    
    
    
    def __call__(self):

        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
      
        while True:
          
            start_time = time()
            
            ret, frame = cap.read()
            assert ret
            
            results = self.predict(frame)
            frame = self.plot_bboxes(results, frame)
            
            end_time = time()
            fps = 1/np.round(end_time - start_time, 2)
             
            cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
            
            cv2.imshow('YOLOv8 Detection', frame)
 
            if cv2.waitKey(5) & 0xFF == 27:
                
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
        
    
detector = ObjectDetection(capture_index='M:/CV/ACC_POC/videos/4.mp4')
detector()

#'M:/CV/ACC_POC/videos/4.mp4'

In [17]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator



model = YOLO("M:/CV/ACC_POC/weights/yolo8n_200epochs.pt")  # load a pretrained YOLOv8n model
model.fuse()

frame1=cv2.imread('571.jpg')
results=model(frame1)
print(results[0])


Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs
Ultralytics YOLOv8.0.38  Python-3.8.16 torch-1.13.1+cpu CPU

0: 640x512 82.2ms
Speed: 0.2ms pre-process, 82.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


tensor([], size=(0, 6))


[]
not one


In [16]:
class_names=model.model.names
print(class_names)

{0: 'helmet', 1: 'no_helmet'}


In [ ]:
def plot_bboxes(self, results, frame):
    
    xyxys = []
    confidences = []
    class_ids = []
    
    # Extract detections for person class
    for result in results[0]:
        class_id = result.boxes.cls.cpu().numpy().astype(int)
        
        if class_id == 0:
            
            xyxys.append(result.boxes.xyxy.cpu().numpy())
            confidences.append(result.boxes.conf.cpu().numpy())
            class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
        
    
    # Setup detections for visualization
    detections = Detections(
                xyxy=results[0].boxes.xyxy.cpu().numpy(),
                confidence=results[0].boxes.conf.cpu().numpy(),
                class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                )
    


# MAIN

In [27]:
import torch
import numpy as np
import cv2
import time
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette,Color
from supervision.tools.detections import Detections, BoxAnnotator

import pyglet #for sound alert

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using Device: ", device)

model = YOLO("M:/CV/ACC_POC/weights/yolo8n_160epochs.pt")  # load a pretrained YOLOv8n model
model.fuse()
CLASS_NAMES_DICT = model.model.names


box_annotator = BoxAnnotator(color=ColorPalette(), thickness=2, text_thickness=2, text_scale=1)

#box_annotator_noHelmet = BoxAnnotator(color=Color(r=255,g=0,b=0), thickness=3, text_thickness=3, text_scale=1.5)
#box_annotator_Helmet = BoxAnnotator(color=Color(r=0,g=255,b=0), thickness=3, text_thickness=3, text_scale=1.5)

def plot_bboxes(results,frame,color_code):
    xyxys = []
    confidences = []
    class_ids = []

    # Extract detections for person class
    for result in results[0]:
        class_id = result.boxes.cls.cpu().numpy().astype(int)
        if class_id == 1:
            xyxys.append(result.boxes.xyxy.cpu().numpy())
            confidences.append(result.boxes.conf.cpu().numpy())
            class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
        
    # Setup detections for visualization
    detections = Detections(
                xyxy=results[0].boxes.xyxy.cpu().numpy(),
                confidence=results[0].boxes.conf.cpu().numpy(),
                class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                )
    
    # Format custom labels
    labels = [f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections]
    
    # Annotate and display frame
    if color_code == 'red':
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
    else:
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
    
    return frame


def sound_alert(frame):
        saved_nohelmetimages_counter=1
        music_voice = pyglet.resource.media('beep_alert_ok.mp3')
        music_voice.play()
        cv2.imwrite('Saved_NoHelmet/'+str(saved_nohelmetimages_counter)+'.jpg',frame)


def stream_vid(capture_index):
    
    res_count=0 
    first_alert=50
    second_alert=200
    third_alert=500
    fourth_alert=1000

    cap = cv2.VideoCapture(capture_index)
    assert cap.isOpened()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while True:
        start_time = time()
        ret, frame = cap.read()
        assert ret

        results = model(frame)
        ##
        xyxy=[]
        class_id=[]
        for result in results[0]:
            xyxy.append(result.boxes.xyxy.cpu().numpy())
            class_id = result.boxes.cls.cpu().numpy().astype(int)
        
        if class_id==1:
            res_count+=1
        else:
            pass
            
        if res_count == first_alert:
            sound_alert(frame)  
        elif res_count == second_alert:
            sound_alert(frame)
        elif res_count == third_alert:
            sound_alert(frame)
        elif res_count == fourth_alert:
            sound_alert(frame)
            res_count=0
        ##

        if class_id == 1:
            frame =  plot_bboxes(results, frame,'red')
            cv2.putText(frame, "Warning!", (50,300), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 4)
        else:
            frame =  plot_bboxes(results, frame,'green')

        end_time = time()
        fps = 1/np.round(end_time - start_time, 2)


        cv2.putText(frame, f'FPS: {int(fps)}', (10,600), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.imshow('YOLOv8 Detection', frame)
        if cv2.waitKey(10) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

capture_index='M:/CV/ACC_POC/videos/2.mp4'
stream_vid(0)


Using Device:  cpu


Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
Ultralytics YOLOv8.0.38  Python-3.8.16 torch-1.13.1+cpu CPU

0: 480x640 154.3ms
Speed: 1.4ms pre-process, 154.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 163.0ms
Speed: 0.0ms pre-process, 163.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 157.3ms
Speed: 0.0ms pre-process, 157.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 155.7ms
Speed: 0.0ms pre-process, 155.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 156.3ms
Speed: 1.1ms pre-process, 156.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 152.7ms
Speed: 1.0ms pre-process, 152.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 145.7ms
Speed: 1.0ms pre-process, 145.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 146.3ms
Speed:

# Backup of MAIN

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

device = 'cpu' if torch.cuda.is_available() else 'cpu'
print("Using Device: ", device)

model = YOLO("M:/CV/ACC_POC/weights/yolo8n_200epochs.pt")  # load a pretrained YOLOv8n model
model.fuse()
CLASS_NAMES_DICT = model.model.names
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)


def plot_bboxes(results,frame):
    xyxys = []
    confidences = []
    class_ids = []
    
    # Extract detections for person class
    for result in results[0]:
        class_id = result.boxes.cls.cpu().numpy().astype(int)
        
        if class_id == 1:
            
            xyxys.append(result.boxes.xyxy.cpu().numpy())
            confidences.append(result.boxes.conf.cpu().numpy())
            class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
        
    # Setup detections for visualization
    detections = Detections(
                xyxy=results[0].boxes.xyxy.cpu().numpy(),
                confidence=results[0].boxes.conf.cpu().numpy(),
                class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                )
    
    # Format custom labels
    labels = [f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections]
    
    # Annotate and display frame
    frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
    
    return frame


def stream_vid(capture_index):
    cap = cv2.VideoCapture(capture_index)
    assert cap.isOpened()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while True:
        start_time = time()
        ret, frame = cap.read()
        assert ret

        results = model(frame)
        frame =  plot_bboxes(results, frame)

        end_time = time()
        fps = 1/np.round(end_time - start_time, 2)

        cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        cv2.imshow('YOLOv8 Detection', frame)
        if cv2.waitKey(10) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



capture_index='M:/CV/ACC_POC/videos/4.mp4'
stream_vid(0)


# Tracker

In [ ]:
def beep_alert():
        beep = pyglet.resource.media('beep_alert.mp3')
        beep.play()

In [ ]:
import cv2
logo=cv2.imread('warning.png')
res=cv2.resize(logo,(100,100))
cv2.imwrite('resized_logo.png',res)